In [31]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import numpy as np
import datetime
import time
import re

In [32]:
chromedriver_dir = r'C:\Users\LG\빅데이터 실습\chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chromedriver_dir)

In [33]:
url = 'https://tv.naver.com/runningman/playlists'

driver.get(url)
time.sleep(3)

In [34]:
# 더보기 계속 진행
for more in np.arange(1, 14):
    xpath = """//*[@id="cds_flick"]/div/div[2]/div/div/div/div/div[3]/a"""
    driver.find_element_by_xpath(xpath).click()

In [35]:
# BeautifulSoup 실행

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup)

<html lang="ko"><head>
<title id="headTitle">
    	
    		네이버 TV
    		
    		
    	
    </title>
<meta charset="utf-8"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex" name="robots"/>
<meta content="summary" name="twitter:card"/>
<meta content="100012287952710" property="fb:admins"/>
<meta content="232491033870086" property="fb:app_id"/>
<meta content="https://tv.naver.com/runningman" property="og:url"/>
<meta content="런닝맨 : 네이버TV" property="og:title"/>
<meta content="[기획의도 by 유재석]
버라이어티 예능이 점점 사라지고 있는 요즘
10년 넘게 일요일 저녁을 책임지고 있는 대한민국 대표 버라이어티 중 하나.
이 프로그램은 수많은 스타와 멤버들이 함께 미션을 하는 것이 특징이며
특히 이 프로그램은 예능의 많은 요소 중 오로지 '웃음'에 집중한다.
" property="og:description"/>
<meta content="website" property="og:type"/>
<meta content="https://phinf.pstatic.net/tvcast/20211201_113/Azn2R_16383232859890e4Wm_PNG/c4bbe97c-359a-4655-af96-d8587c617efb.png?typ

In [36]:
# 저장할 데이터 프레임
runningman_view = pd.DataFrame(columns=("회차", "영상 이름", "조회수", "좋아요 수"))
runningman_view

,회차,영상 이름,조회수,좋아요 수


In [37]:
# 재생 목록 이름만 가져옴
# 회차에 해당하는 리스트만 가져옴
playlist_names = []
playlists = soup.find_all('div', 'playlist _MM_CARD')
# 재생 목록 이름 리스트

tmp = -1
for playlist in playlists:
    tmp += 1
    if (tmp < 5): continue
    playlist_name = playlist.find('h3', 'play_tit').get_text() # 재생 목록 이름 가져옴
    playlist_name = playlist_name.replace("  ", " ")
    playlist_name = playlist_name[:8].strip() # 재생 목록을 프로그램 이름 + 회차까지 그리고 공백 제거
    if tmp < 7:
        playlist_name = playlist_name[:7] + "회"
    playlist_names.append(playlist_name)
    
    if (tmp == 58): # 아는 형님 결과 수랑 동일하게 지정
        break
        
print(playlist_names)

['런닝맨 579회', '런닝맨 578회', '런닝맨 577회', '런닝맨 576회', '런닝맨 575회', '런닝맨 574회', '런닝맨 573회', '런닝맨 572회', '런닝맨 571회', '런닝맨 570회', '런닝맨 569회', '런닝맨 568회', '런닝맨 567회', '런닝맨 566회', '런닝맨 565회', '런닝맨 564회', '런닝맨 563회', '런닝맨 562회', '런닝맨 561회', '런닝맨 560회', '런닝맨 559회', '런닝맨 558회', '런닝맨 557회', '런닝맨 556회', '런닝맨 555회', '런닝맨 554회', '런닝맨 553회', '런닝맨 552회', '런닝맨 551회', '런닝맨 550회', '런닝맨 549회', '런닝맨 548회', '런닝맨 547회', '런닝맨 546회', '런닝맨 545회', '런닝맨 544회', '런닝맨 543회', '런닝맨 542회', '런닝맨 541회', '런닝맨 540회', '런닝맨 539회', '런닝맨 538회', '런닝맨 537회', '런닝맨 536회', '런닝맨 535회', '런닝맨 534회', '런닝맨 533회', '런닝맨 532회', '런닝맨 531회', '런닝맨 530회', '런닝맨 529회', '런닝맨 528회', '런닝맨 527회', '런닝맨 526회']


579회 ~ 526회

In [38]:
len(playlist_names)

54

In [39]:
playlists = soup.find_all('div', 'playlist _MM_CARD')
# 재생 목록 이름 리스트
i = -1
for playlist in playlists:  
    i += 1
    if (i < 5): continue
    if (i == 59): break
    
    playlist_name_list = [] # 재생 목록 이름 저장할 리스트
    vedio_name_list = [] # 영상 이름 저장할 리스트
    view_count = [] # 조회수 저장할 리스트
    like_count = [] # 좋아요 수 저장할 리스트
        
    vedio_names = playlist.find_all("a") # 영상 이름
    for vedio_name in vedio_names:
        vedio_name_tmp = vedio_name.select_one("dl dt a tooltip")
        if (vedio_name_tmp != None):
            vedio_name_tmp = vedio_name.select_one("dl dt a tooltip")['title'].strip()
            vedio_name_list.append(vedio_name_tmp)
    
    name_tmp = playlist_names[i-5]
    view_results = playlist.find_all('span', 'hit') # 조회수들 가져옴
    for view_result in view_results:
        view_result = view_result.get_text()
        view = int(view_result[4:].replace(',', '')) # 조회수 글자제외, int형 변환
        view_count.append(view) # 리스트에 조회수 저장
        playlist_name_list.append(name_tmp)
            
    like_results = playlist.find_all('span', 'like') # 좋아요 수 가져옴
    for like_result in like_results:
        like_result = like_result.get_text().strip()
        like = int(like_result[5:].replace(',', '')) # 좋아요 수 글자제외, int형 변환
        like_count.append(like) # 리스트에 좋아요 수 저장
    
    # 데이터 프레임 생성
    df = pd.DataFrame([x for x in zip(playlist_name_list, vedio_name_list, view_count, like_count)],
                        columns=("회차", "영상 이름", "조회수", "좋아요 수"))
    # 앞에서 만들어 둔 데이터 프레임에 추가
    runningman_view = runningman_view.append(df)

In [40]:
runningman_view

,회차,영상 이름,조회수,좋아요 수
0,런닝맨 579회,"[선공개] 아이키, 잔망 넘치는 오프닝 직캠♥",51336,482
1,런닝맨 579회,"[선공개] 허니제이, 절도 있는 카리스마 오프닝 직캠♥",134396,1950
2,런닝맨 579회,"[선공개] 리정, 클래스가 다른 박력 오프닝 직캠♥",70370,683
3,런닝맨 579회,"[선공개] 모니카, 우아함 가득 오프닝 직캠♥",39267,437
4,런닝맨 579회,"지석진, 허니제이 인터뷰에 토크 트라우마 재발",36826,323
...,...,...,...,...
9,런닝맨 526회,"“말을 해야 될 거 아냐” 김종국, 같은팀 이광수 이름표 떼고 안타깝게 아웃!",12162,119
10,런닝맨 526회,"[현웃 주의] ‘유령’ 임원희, 신선한 이름표 떼기로 송지효 아웃!(ft. 타이밍 ...",17382,155
11,런닝맨 526회,"이제훈×양세찬, 작전 이해 못한 임원희에 폭풍 잔소리♨",16065,151
12,런닝맨 526회,"‘변절자’ 지석진, 시원한 물벼락 폭탄에 입틀막!",13898,113


In [29]:
# 기존
runningman_view

,회차,영상 이름,조회수,좋아요 수
0,런닝맨 579회,"[선공개] 아이키, 잔망 넘치는 오프닝 직캠♥",43813,440
1,런닝맨 579회,스휴파,601,14
2,런닝맨 579회,"[선공개] 허니제이, 절도 있는 카리스마 오프닝..",125826,1889
3,런닝맨 579회,"[선공개] 리정, 클래스가 다른 박력 오프닝 직..",64535,654
4,런닝맨 579회,"[선공개] 모니카, 우아함 가득 오프닝 직캠♥",37307,421
...,...,...,...,...
9,런닝맨 526회,"“말을 해야 될 거 아냐” 김종국, 같은팀 이광수..",12152,119
10,런닝맨 526회,"[현웃 주의] ‘유령’ 임원희, 신선한 이름표 떼..",17367,155
11,런닝맨 526회,"이제훈×양세찬, 작전 이해 못한 임원희에 폭풍 ..",16051,151
12,런닝맨 526회,"‘변절자’ 지석진, 시원한 물벼락 폭탄에 입틀막..",13886,113


In [41]:
# csv 파일로 저장
# 2021-11-17 11:55 저장
runningman_view.to_csv('runningman_view_name.csv', index=False, encoding="utf-8-sig")

In [42]:
save_test = pd.read_csv('runningman_view_name.csv')
save_test

,회차,영상 이름,조회수,좋아요 수
0,런닝맨 579회,"[선공개] 아이키, 잔망 넘치는 오프닝 직캠♥",51336,482
1,런닝맨 579회,"[선공개] 허니제이, 절도 있는 카리스마 오프닝 직캠♥",134396,1950
2,런닝맨 579회,"[선공개] 리정, 클래스가 다른 박력 오프닝 직캠♥",70370,683
3,런닝맨 579회,"[선공개] 모니카, 우아함 가득 오프닝 직캠♥",39267,437
4,런닝맨 579회,"지석진, 허니제이 인터뷰에 토크 트라우마 재발",36826,323
...,...,...,...,...
744,런닝맨 526회,"“말을 해야 될 거 아냐” 김종국, 같은팀 이광수 이름표 떼고 안타깝게 아웃!",12162,119
745,런닝맨 526회,"[현웃 주의] ‘유령’ 임원희, 신선한 이름표 떼기로 송지효 아웃!(ft. 타이밍 ...",17382,155
746,런닝맨 526회,"이제훈×양세찬, 작전 이해 못한 임원희에 폭풍 잔소리♨",16065,151
747,런닝맨 526회,"‘변절자’ 지석진, 시원한 물벼락 폭탄에 입틀막!",13898,113


In [43]:
save_test.groupby('회차').count()

,영상 이름,조회수,좋아요 수
회차,,,
런닝맨 526회,14,14,14
런닝맨 527회,15,15,15
런닝맨 528회,13,13,13
런닝맨 529회,15,15,15
런닝맨 530회,14,14,14
런닝맨 531회,14,14,14
런닝맨 532회,12,12,12
런닝맨 533회,14,14,14
런닝맨 534회,14,14,14


In [44]:
len(save_test.groupby('회차').count().index)

54